**Data used:** https://www.kaggle.com/datasets/grassknoted/asl-alphabet/data

In [1]:
!pip install mediapipe==0.10.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.14.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.


In [2]:
import mediapipe as mp
mp_hands = mp.solutions.hands
print("Success! You are on the safe version.")

Success! You are on the safe version.


In [3]:
import kagglehub
import os

print("Downloading dataset... this may take a minute.")
# Download latest version
dataset_path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Dataset downloaded to:", dataset_path)

# AUTOMATICALLY FIND THE TRAINING FOLDER
# The dataset structure is often: path -> asl_alphabet_train -> asl_alphabet_train -> A, B, C...
DATA_DIR = None

# Walk through the download to find the folder that actually contains 'A', 'B', etc.
for root, dirs, files in os.walk(dataset_path):
    if "A" in dirs and "B" in dirs:
        DATA_DIR = root
        break

if DATA_DIR:
    print(f"SUCCESS: Training data found at: {DATA_DIR}")
else:
    print("ERROR: Could not verify dataset structure. Please check the path manually.")

100%|██████████| 1.03G/1.03G [00:35<00:00, 31.4MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1
SUCCESS: Training data found at: /root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_train/asl_alphabet_train


In [4]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Initialize MediaPipe
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

data = []
labels = []

# Verify DATA_DIR exists before starting
if not DATA_DIR or not os.path.exists(DATA_DIR):
    print("Error: DATA_DIR is invalid. Run the previous cell again.")
else:
    print(f"Processing images from: {DATA_DIR}")

    # Get list of classes (A, B, C...)
    classes = sorted(os.listdir(DATA_DIR))

    for dir_ in classes:
        # Construct path to the class folder (e.g., .../A)
        class_dir = os.path.join(DATA_DIR, dir_)

        # Skip if it's not a folder
        if not os.path.isdir(class_dir):
            continue

        print(f"Processing Class: {dir_}")

        # Loop through images in that class
        # (Optional: Add [:1000] to os.listdir(...) if you want to limit data for testing)
        for img_path in os.listdir(class_dir):
            data_aux = []
            x_ = []
            y_ = []

            path_to_img = os.path.join(class_dir, img_path)
            img = cv2.imread(path_to_img)

            if img is None:
                continue

            # 1. Add Padding (Crucial for this dataset)
            # The hands are huge and touch edges. 80px padding helps MediaPipe see them.
            h, w, c = img.shape
            img = cv2.copyMakeBorder(img, 80, 80, 80, 80, cv2.BORDER_CONSTANT, value=[0, 0, 0])

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            if results.multi_hand_landmarks:
                # OPTIMIZATION: Only take the FIRST detected hand.
                # This prevents "84 feature" errors if a shadow looks like a 2nd hand.
                hand_landmarks = results.multi_hand_landmarks[0]

                # Collect coordinates
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    x_.append(x)
                    y_.append(y)

                # Normalize
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

                # Save if valid
                if len(data_aux) == 42:
                    data.append(data_aux)
                    labels.append(dir_)

    print(f"Done! Collected {len(data)} samples.")

    # Save the file
    f = open("data.pickle", "wb")
    pickle.dump({"data": data, "labels": labels}, f)
    f.close()

    print("data.pickle has been created successfully.")

Processing images from: /root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_train/asl_alphabet_train
Processing Class: A


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing Class: B
Processing Class: C
Processing Class: D
Processing Class: E
Processing Class: F
Processing Class: G
Processing Class: H
Processing Class: I
Processing Class: J
Processing Class: K
Processing Class: L
Processing Class: M
Processing Class: N
Processing Class: O
Processing Class: P
Processing Class: Q
Processing Class: R
Processing Class: S
Processing Class: T
Processing Class: U
Processing Class: V
Processing Class: W
Processing Class: X
Processing Class: Y
Processing Class: Z
Processing Class: del
Processing Class: nothing
Processing Class: space
Done! Collected 68320 samples.
data.pickle has been created successfully.


In [5]:
# If running on Google Colab:
try:
    from google.colab import files
    files.download('data.pickle')
except ImportError:
    print("Not running on Colab. Look for 'data.pickle' in your Output file browser on the left.")

# If running on Kaggle:
# The file is now in the "Output" directory (usually /kaggle/working/).
# You can manually download it from the sidebar menu on the right/left.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>